In [1]:
import pandas as pd
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
import numpy as np
import matplotlib.pyplot as plt 
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import plotly.graph_objects as go
from plotly_resampler import FigureResampler, FigureWidgetResampler

In [2]:
df = pd.read_parquet("../data/static_models/pump1_power_siso.par")
df = df.set_index("time")
df = df.rolling(window=40, center=True).median().dropna()
df = df.resample("5s").mean().dropna()

In [3]:
df['pump1_speed'] = df['pump1_speed'].apply(lambda x: max(0, x))


In [4]:
train_df = df["2024-01-10 06:00:00":"2024-01-10 11:00:00"]
test_df = df["2024-01-11":"2024-02-16 12:00:00"]

In [5]:
from src.system_identification import ARXParametersIdentification
from src.preprocessing import Preprocessor
import pandas as pd
from src.sysidentpy.basis_function._basis_function import Polynomial

In [6]:
FEATURES = "pump1_speed"
TARGET = 'pump1_power'

sys_id = ARXParametersIdentification(FEATURES, 
                            TARGET, 
                            dataframe= train_df, 
                            model_type= 'siso')

In [7]:
parameters = {
    'estimator': 'ridge_regression',
    'ridge_param': 2e4,
    'order_selection': False,
    'n_terms': 3,
    'extended_least_squares': False,
    'ylag': 2,  
    'xlag': 1, 
    'basis_function': Polynomial(degree=1)
}
    
fitted_model, model_parameters = sys_id.identify_arx_parameters(x_train=train_df[FEATURES], 
                                                                y_train= train_df[TARGET],
                                                                **parameters)
    
model_parameters

/home/alqua/Git/edge_mpc/sys_identification/src/sysidentpy/narmax_base.py:1034: RuntimeWarning: divide by zero encountered in divide
  x = x[1:] / aux_b
/home/alqua/Git/edge_mpc/sys_identification/src/sysidentpy/narmax_base.py:1034: RuntimeWarning: invalid value encountered in divide
  x = x[1:] / aux_b


,Regressors,Parameters,ERR
0,x1(k-1),3.5117E-02,9.9897E-01
1,y(k-1),1.6108E-01,NAN
2,y(k-2),9.5030E-02,NAN


In [8]:
yhat  = sys_id.predict(x_test= test_df[FEATURES], y_test= test_df[TARGET], n_steps_ahead= 200)

val_plots = sys_id.residuals_analysis(yhat, test_df[TARGET].values)
val_plots
print(model_parameters)

RRSE: 0.106177021389858
MSE: 7.488975628814937
  Regressors  Parameters         ERR
0    x1(k-1)  3.5117E-02  9.9897E-01
1     y(k-1)  1.6108E-01         NAN
2     y(k-2)  9.5030E-02         NAN


In [9]:
test_df["yhat"] = yhat

sys_id.plot_features_and_target(test_df, ['yhat', 'pump1_power'])

FigureWidgetResampler({
    'data': [{'name': '<b style="color:sandybrown">[R]</b> yhat <i style="color:#fc9944">~53m</i>',
              'showlegend': True,
              'type': 'scattergl',
              'uid': 'aebf6f4e-12cc-4cde-a574-a512354081ff',
              'x': array([datetime.datetime(2024, 1, 11, 0, 0, tzinfo=<UTC>),
                          datetime.datetime(2024, 1, 11, 0, 33, 30, tzinfo=<UTC>),
                          datetime.datetime(2024, 1, 11, 1, 20, 45, tzinfo=<UTC>), ...,
                          datetime.datetime(2024, 2, 16, 9, 41, 40, tzinfo=<UTC>),
                          datetime.datetime(2024, 2, 16, 11, 18, 25, tzinfo=<UTC>),
                          datetime.datetime(2024, 2, 16, 11, 26, 45, tzinfo=<UTC>)], dtype=object),
              'y': array([50.11000023, 47.49104719, 50.98426052, ..., 47.44368687, 49.56801039,
                          48.1518016 ])},
             {'name': ('<b style="color:sandybrown">[R' ... 'style="color:#fc9944">~53m</i>'